Chiara Barbieri, 517096 --
Bachelor's thesis

# <center> Clean models <center>


# Libraries

In [ ]:
pip install boto3 parallel_wavegan munch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 155.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.7 MB/s eta 0:00:00
  Created wheel for parallel_wavegan: filename=parallel_wavegan-0.6.1-py3-none-any.whl size=106235 sha256=64a07d386b98c140e3bb2ac541ee5c3ccdeefa56420d79bf7219bbf25a855076
  Stored in directory: /root/.cache/pip/wheels/fe/4e/98/fbd8f6edcc796c3e785f94c81e56f71e3c3333248e0402ba13
Successfully built parallel_wavegan


In [ ]:
"""Libraries to managae directories"""
import os
import gdown
import traceback
import sys
import importlib
from pathlib import Path
from zipfile import ZipFile

"""Libraries to manage data"""
import math
import random
import argparse
import time
from tqdm import tqdm
from collections import Counter
from typing import List, Tuple, Dict, Optional, Callable

"""Libraries to manage datasets"""
import json
import tarfile
import subprocess
import pickle
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

"""Libraries to manage audios"""
import boto3
import soundfile as sf
import click
import yaml
import glob
import tensorboard
import parallel_wavegan
import pydub

import transformers
from transformers import AutoModel

import munch
from munch import Munch

"""Scipy"""
import scipy
from scipy.spatial.distance import cdist
from scipy import signal
from scipy.io import wavfile

"""Torchaudio"""
import torchaudio
import torchaudio.transforms as T
from torchaudio.functional import resample

"""Pytorch"""
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm, remove_weight_norm
import torch.optim as optim

"""Librosa"""
import librosa
import librosa.display
from librosa.display import waveshow, specshow
import librosa.feature
import librosa.feature.inverse
import librosa.effects

"""IPython"""
import IPython
from IPython import display
from IPython.display import Audio, display

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Don't run the following cell if not on Google Collaboratory:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change to the thesis directory
os.chdir('/content/drive/MyDrive/Thesis')
print("Current directory:", os.getcwd())

Mounted at /content/drive
Current directory: /content/drive/MyDrive/Thesis


# Importing helper functions

In [ ]:
try:
    import featureExtraction_MFCCs
    from featureExtraction_MFCCs import *
    # To reload the module if "featureExtraction_MFCCs.py" has been updated
    importlib.reload(featureExtraction_MFCCs)

    import wav_helperFunctions
    from wav_helperFunctions import *
    # To reload the module if "wav_helperFunctions.py" has been updated
    importlib.reload(wav_helperFunctions)

    print("All helper functions are imported")
except:
    os.chdir('/content/drive/MyDrive/Thesis/helper_functions')
    print("Current directory:", os.getcwd())

    from wav_helperFunctions import load_npz, plot_waveform, play_audio
    from featureExtraction_MFCCs import extract_mfcc, play_mfcc, plot_fullMFCC, plot_restrictedMFCC

    print("All helper functions are imported")

Current directory: /content/drive/MyDrive/Thesis/helper_functions
All helper functions are imported


# Importing the clean datasets

Import of the clean datasets that were saved in the notebook "Import dataset and preprocessing"

In [ ]:
# Importing the (clean) preprocessed GCSD-10 dataset
GSCD10_path = r"d:\\Bachelor AI\\Thesis\\Coding\\GSCD10_clean.npz"
GSCD10_dpath = "/content/drive/MyDrive/Thesis/GSCD10_clean.npz"

try:
  # If the code is running on the local device
  GSCD10 = load_npz(GSCD10_path)
except:
  # If the code is running on Google Collab
  GSCD10 = load_npz(GSCD10_dpath)

# We have a list of dictionaries (each sample has its dictionary)
for key in GSCD10[0].keys():
    arr = GSCD10[0][key]
    print(f"{key}: type={type(arr)}")

waveform: type=<class 'numpy.ndarray'>
sr: type=<class 'int'>
label: type=<class 'int'>
path: type=<class 'str'>


In [ ]:
# Importing the (clean) preprocessed GCSD-30 dataset
GSCD30_path = r"d:\\Bachelor AI\\Thesis\\Coding\\GSCD30_clean.npz"
GSCD30_dpath = "/content/drive/MyDrive/Thesis/GSCD30_clean.npz"

try:
  # If the code is running on the local device
  GSCD30 = load_npz(GSCD30_path)
except:
  # If the code is running on Google Collab
  GSCD30 = load_npz(GSCD30_dpath)

# We have a list of dictionaries (each sample has its dictionary)
for key in GSCD30[0].keys():
    arr = GSCD30[0][key]
    print(f"{key}: type={type(arr)}")

waveform: type=<class 'numpy.ndarray'>
sr: type=<class 'int'>
label: type=<class 'int'>
path: type=<class 'str'>


# Preparing data for training

## Extracting the MFCCs features

In [ ]:
def convert_to_mfcc_format(dataset_list):
    if len(dataset_list) == 0:
        return {
            'waveforms': [],
            'labels': [],
            'files': [],
            'sample_rate': 16000
        }

    return {
        'waveforms': [sample['waveform'] for sample in dataset_list],
        'labels': [sample['label'] for sample in dataset_list],
        'files': [sample['path'] for sample in dataset_list],
        'sample_rate': dataset_list[0]['sr']
    }

In [ ]:
GSCD10 = convert_to_mfcc_format(GSCD10)
GSCD30 = convert_to_mfcc_format(GSCD30)

Here we extract the MFCCs features: 40 Mel-frequency cepstral coefficient (MFCCs) bands using a window length of 25 ms and a hop size of 10 ms.

In [ ]:
GSCD10_mfcc = extract_mfcc(dataset = GSCD10,
                            n_mfcc = 40,                 # 40 mel-bands
                            step = 0.01,                 # step of 10ms
                            window_length = 0.025,       # window length of 25ms
)

In [ ]:
GSCD30_mfcc = extract_mfcc(dataset = GSCD30,
                            n_mfcc = 40,                 # 40 mel-bands
                            step = 0.01,                 # step of 10ms
                            window_length = 0.025,       # window length of 25ms
)

## Padding

Padding is used in order to have all the MFCCs tensor of the same length.

In [ ]:
def pad_sequence(dataset):
    """Pad all MFCCs in dataset['MFCCs'] to the same length"""

    mfcc_list = []
    for mfcc in dataset['MFCCs']:
        if isinstance(mfcc, np.ndarray):
            mfcc_list.append(torch.from_numpy(mfcc).float())
        else:
            mfcc_list.append(mfcc.float())

    # Apply padding
    mfcc_list = [item.t() for item in mfcc_list]
    mfcc_list = torch.nn.utils.rnn.pad_sequence(mfcc_list, batch_first=True, padding_value=0.)
    mfcc_list = mfcc_list.permute(0, 2, 1)

    dataset['MFCCs'] = mfcc_list

    return dataset

In [ ]:
# Padding the samples to have the same input length
GSCD10_mfcc = pad_sequence(GSCD10_mfcc)
GSCD30_mfcc = pad_sequence(GSCD30_mfcc)

## Splitting of the data

The papers use an 80/20 split, meaning that they first split the dataset into training set (80%) and test set (20%), and then they split again the training set into train set (80%) and validation set (20%). To be have reproducibility, which samples are in which set is defined in the lists "validation_list.txt" and "testing_list.txt" (with the remaining samples being in the test set).

In [ ]:
class SubsetSC():
    def __init__(
            self,
            dataset: dict,
            official_split_root: str = None
        ):

        self.dataset = dataset

        # Load official split files:
        val_list_path = os.path.join(official_split_root, "validation_list.txt")
        test_list_path = os.path.join(official_split_root, "testing_list.txt")

        with open(val_list_path, 'r') as f:
            val_files = set(line.strip() for line in f)

        with open(test_list_path, 'r') as f:
            test_files = set(line.strip() for line in f)

        # Match paths to splits
        self.train_idx = []
        self.val_idx = []
        self.test_idx = []

        for i, path in enumerate(dataset['files']):
            # Extract relative path
            normalized_path = path.replace('\\', '/')
            parts = normalized_path.split('/')[-2:]
            rel_path = '/'.join(parts)

            if rel_path in test_files:
                self.test_idx.append(i)
            elif rel_path in val_files:
                self.val_idx.append(i)
            else:
                self.train_idx.append(i)

        self.train_idx = np.array(self.train_idx)
        self.val_idx = np.array(self.val_idx)
        self.test_idx = np.array(self.test_idx)

    def get_split(self, split: str = "train"):

        if split == 'train':
            idx = self.train_idx
        elif split == 'val':
            idx = self.val_idx
        elif split == 'test':
            idx = self.test_idx
        # Constraint on split name
        else:
            raise ValueError(f"Invalid split: {split}. Must be 'train', 'val', or 'test'")

        # Create new dictionary with selected indices
        return {
            'MFCCs': [self.dataset['MFCCs'][i] for i in idx],
            'labels': [self.dataset['labels'][i] for i in idx],
            'files': [self.dataset['files'][i] for i in idx],
            'mapping': self.dataset['mapping']
        }

In [ ]:
path_win = r"d:\\Bachelor AI\\Thesis\\Coding\\speech_commands_v0.01"
path_colab = r"/content/drive/MyDrive/Thesis/speech_commands_v0.01"

if os.path.exists(path_win):
    speech_commands_root = path_win
elif os.path.exists(path_colab):
    speech_commands_root = path_colab
else:
    raise FileNotFoundError(
        f"Neither directory exists:\n - {path_win}\n - {path_colab}"
    )

print(f"Using: {speech_commands_root}")

files_and_folders = os.listdir(speech_commands_root)
print(files_and_folders)


Using: /content/drive/MyDrive/Thesis/speech_commands_v0.01
['seven', 'house', 'right', 'down', 'go', 'zero', 'six', 'wow', 'happy', 'bird', 'three', 'marvin', 'stop', 'five', 'two', 'on', 'off', 'four', 'dog', 'up', 'tree', 'bed', 'cat', 'left', 'LICENSE', 'validation_list.txt', 'testing_list.txt', 'eight', 'sheila', '_background_noise_', 'nine', 'yes', 'one', 'no', 'README.md']


### Splitting the data for GSCD-10

In [ ]:
len(GSCD10_mfcc['MFCCs'])

21312

In [ ]:
split_10 = SubsetSC(GSCD10_mfcc, speech_commands_root)

train_data10 = split_10.get_split('train')
print("Number of samples in the training data: ", len(train_data10["MFCCs"]))

val_data10 = split_10.get_split('val')
print("Number of samples in the validation data: ", len(val_data10["MFCCs"]))

test_data10 = split_10.get_split('test')
print("Number of samples in the testing data: ", len(test_data10["MFCCs"]))

Number of samples in the training data:  16624
Number of samples in the validation data:  2325
Number of samples in the testing data:  2363


### Splitting the data for GSCD-30

In [ ]:
len(GSCD30_mfcc['MFCCs'])

58252

In [ ]:
split_30 = SubsetSC(GSCD30_mfcc, speech_commands_root)

train_data30 = split_30.get_split('train')
print("Number of samples in the training data: ", len(train_data30["MFCCs"]))

val_data30 = split_30.get_split('val')
print("Number of samples in the validation data: ", len(val_data30["MFCCs"]))

test_data30 = split_30.get_split('test')
print("Number of samples in the testing data: ", len(test_data30["MFCCs"]))

Number of samples in the training data:  45884
Number of samples in the validation data:  6137
Number of samples in the testing data:  6231


## Preparing the data to pass in the dataloader

A dataloader only accepts a Dataset object that implements __getitem__ and __len__, so the current dictionaries in which the data is stored cannot be used.

In [ ]:
class prepareDataset(Dataset):
    def __init__(self, data_dict):
        self.mfccs = data_dict['MFCCs']
        self.labels = data_dict['labels']
        self.files = data_dict['files']
        self.mapping = data_dict['mapping']

    def __len__(self):
        return len(self.mfccs)

    def __getitem__(self, idx):
        """
        Returns:
            mfcc: The MFCC features (as tensor)
            label: The corresponding label (as tensor)
        """
        mfcc = torch.FloatTensor(self.mfccs[idx])
        label = torch.LongTensor([self.labels[idx]])[0]  # Convert to scalar tensor

        return mfcc, label

In [ ]:
train_dataset10 = prepareDataset(train_data10)
val_dataset10 = prepareDataset(val_data10)
test_dataset10 = prepareDataset(test_data10)

In [ ]:
train_dataset30 = prepareDataset(train_data30)
val_dataset30 = prepareDataset(val_data30)
test_dataset30 = prepareDataset(test_data30)

## Dataloaders

In [ ]:
# Parameters for the dataloaders
use_cuda = torch.cuda.is_available()

batch_size = 64 if use_cuda else 32
num_workers = 4 if use_cuda else 0
pin_memory = use_cuda

print(f"batch_size: {batch_size}")
print(f"num_workers: {num_workers}")
print(f"pin_memory: {pin_memory}")

batch_size: 64
num_workers: 4
pin_memory: True


### Train set dataloader

In [ ]:
train_loader10 = DataLoader(
    train_dataset10,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
train_loader10

In [ ]:
train_loader30 = DataLoader(
    train_dataset30,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
train_loader30

### Validation set datalaoder

In [ ]:
val_loader10 = DataLoader(
    val_dataset10,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
val_loader10

In [ ]:
val_loader30 = DataLoader(
    val_dataset30,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
val_loader30

### Test set dataloader

In [ ]:
test_loader10 = DataLoader(
    test_dataset10,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
test_loader10

In [ ]:
test_loader30 = DataLoader(
    test_dataset30,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
test_loader30

# Defining the models

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Simple CNN
*from the article "Adversial Example detection by classification for deep speech recognition"*

In [ ]:
class simpleCNN(nn.Module):
    def __init__(
            self,
            input_shape,
            n_output = 10    # Number of possible labels
        ):
        super(simpleCNN, self).__init__()

        # Store input shape for reference
        self.input_shape = input_shape

        # 1st conv layer -- activation: ReLU
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(2, 2)) # L2 kernel regularizer during the training process
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 3))

        # 2nd conv layer -- activation: ReLU
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(2, 2)) # L2 kernel regularizer during the training process
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # 3rd conv layer -- activation: SeLU
        self.conv3 = nn.Conv2d(64, 32, kernel_size=(2, 2)) # L2 kernel regularizer during the training process
        self.bn3 = nn.BatchNorm2d(32)
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        self.dropout1 = nn.Dropout(0.4)

        # Fully connected layers -- activation: ReLU and softmax
        self.flattened_size = self._get_flattened_size(input_shape)

        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, n_output)

    def _get_flattened_size(self, input_shape):
        """Calculate the size after conv and pooling layers"""
        with torch.no_grad():

            # Constraint on input shape
            if len(input_shape) == 2:
                h, w = input_shape # Ensure input_shape is (height, width)
            else:
                raise ValueError(f"Expected 2D input_shape, got shape: {input_shape}")

            dummy_input = torch.zeros(1, 1, h, w)

            x = self.conv1(dummy_input)
            x = F.relu(self.bn1(x))
            x = self.pool1(x)

            x = self.conv2(x)
            x = F.relu(self.bn2(x))
            x = self.pool2(x)

            x = self.conv3(x)
            x = F.selu(self.bn3(x))
            x = self.pool3(x)

            # Get flatten size
            flattened_size = x.view(1, -1).size(1)

            return flattened_size

    def forward(self, x):

        # Ensure input has channel dimension: (batch, channels, height, width)
        if x.dim() == 3:
            x = x.unsqueeze(1)

        # Conv block 1 (with ReLU)
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)

        # Conv block 2 (with ReLU)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)

        # Conv block 3 (with SeLU)
        x = self.conv3(x)
        x = F.selu(self.bn3(x))
        x = self.pool3(x)
        x = self.dropout1(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Dense layers
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

## Complex CNN
*from the article "Trojaning attack on Neural Networks"*

In [ ]:
class complexCNN(nn.Module):
    def __init__(
            self,
            input_shape,
            n_output = 10    # Number of possible labels
        ):
        super(complexCNN, self).__init__()

        # 1st conv layer -- activation: None
        self.conv1 = nn.Conv2d(1, 96, kernel_size=(3, 3), padding=1) # L2 kernel regularizer during the training process
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))

        # 2nd conv layer -- activation: None
        self.conv2 = nn.Conv2d(96, 256, kernel_size=(3, 3), padding=1) # L2 kernel regularizer during the training process
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # 3rd conv layer -- activation: ReLU
        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=1) # L2 kernel regularizer during the training process

        # 4th conv layer -- activation: ReLU
        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3, 3), padding=1) # L2 kernel regularizer during the training process

        # 5th conv layer -- activation: ReLU
        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=1) # L2 kernel regularizer during the training process
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1)

        # Dense layers -- activation: ReLU
        self.flattened_size = self._get_flattened_size(input_shape)
        self.fc1 = nn.Linear(self.flattened_size, 256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, n_output)

    def _get_flattened_size(self, input_shape):
        """Calculate the size after conv and pooling layers"""
        with torch.no_grad():

            # Constraint on input shape
            if len(input_shape) == 2:
                h, w = input_shape # Ensure input_shape is (height, width)
            else:
                raise ValueError(f"Expected 2D input_shape, got shape: {input_shape}")

            dummy_input = torch.zeros(1, 1, h, w)

            x = self.conv1(dummy_input)
            x = self.pool1(x)

            x = self.conv2(x)
            x = self.pool2(x)

            x = F.relu(self.conv3(x))
            x = F.relu(self.conv4(x))
            x = F.relu(self.conv5(x))
            x = self.pool3(x)

            # Get flatten size
            flattened_size = x.view(1, -1).size(1)

            return flattened_size

    def forward(self, x):

        # Conv block 1 (no activation)
        x = self.conv1(x)
        x = self.pool1(x)

        # Conv block 2 (no activation)
        x = self.conv2(x)
        x = self.pool2(x)

        # Conv blocks 3-5 (with ReLU)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.pool3(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Dense layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

## Speech-specific LSTM
*from the article "A neural attention model for speech command recognition"*

In [ ]:
class speechSpecificLSTM(nn.Module):
    def __init__(
            self,
            n_output=10,
            lstm_hidden=64,
            attention_dim=128
        ):
        super(speechSpecificLSTM, self).__init__()

        # 1st conv layer -- activation: ReLU
        self.conv1 = nn.Conv2d(1, 10, kernel_size=(5, 1)) # (5,1) kernels for temporal processing
        self.bn1 = nn.BatchNorm2d(10)

        # 2nd conv layer -- activation: ReLU
        self.conv2 = nn.Conv2d(10, 1, kernel_size=(5, 1)) # (5,1) kernels for temporal processing
        self.bn2 = nn.BatchNorm2d(1)

        # Bidirectional LSTM layers: capture 2-way long term dependencies in the audio file
        self.lstm1 = nn.LSTM(
            input_size = 40,              # Input: 40-mel bands
            hidden_size = lstm_hidden,
            num_layers = 1,
            batch_first = True,
            bidirectional = True
        )

        self.lstm2 = nn.LSTM(
            input_size = lstm_hidden * 2,  # *2 because bidirectional
            hidden_size = lstm_hidden,
            num_layers = 1,
            batch_first = True,
            bidirectional = True
        )

        # Attention mechanism: query projection from last hidden state
        self.query_proj = nn.Linear(lstm_hidden * 2, attention_dim)

        # Dense layers after attention
        self.fc1 = nn.Linear(lstm_hidden * 2, 64) # Uses ReLU activation
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 32) # Uses ReLU activation
        self.fc3 = nn.Linear(32, n_output) # Uses Softmax activation

    def attention(self, query, keys_values):
        """Dot-product attention mechanism"""

        # Expand query
        query_expanded = query.unsqueeze(1)

        # Compute attention scores using the dot product (matrix-matrix product)
        att_scores = torch.bmm(query_expanded, keys_values.transpose(1, 2))

        # Remove middle dimension
        att_scores = att_scores.squeeze(1)

        # Apply softmax
        att_weights = F.softmax(att_scores, dim=1)
        att_weights_expanded = att_weights.unsqueeze(1)

        # Compute the most relevant part of the audio
        context = torch.bmm(att_weights_expanded, keys_values)
        context = context.squeeze(1)

        return context, att_weights

    def forward(self, x):

        # Prepare input
        x = x.transpose(2, 3)

        # Conv block 1
        x = F.relu(self.conv1(x))
        x = self.bn1(x)

        # Conv block 2
        x = F.relu(self.conv2(x))
        x = self.bn2(x)

        # Squeeze channel dimension
        x = x.squeeze(1)

        # First Bidirectional LSTM
        x, _ = self.lstm1(x)
        # Second Bidirectional LSTM
        x, _ = self.lstm2(x)

        # Get last timestep for query
        x_last = x[:, -1, :]

        # Project to query space
        query = self.query_proj(x_last)
        # Apply attention mechanism
        context, att_weights = self.attention(query, x)

        # Dense layers
        x = F.relu(self.fc1(context))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

## Original LSTM
*from the article "Long Short-Term memory"*

In [ ]:
class originalLSTM(nn.Module):
    def __init__(
            self,
            hidden_size=64,
            num_layers=2,
            n_output=10,
            dropout=0.4
        ):
        super(originalLSTM, self).__init__()

        # Set parameters
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Preprocessing layer
        self.bn = nn.BatchNorm2d(1)

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size = 40,            # 40-mel bands
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True,
            dropout = dropout if num_layers > 1 else 0.0
        )

        # Output dense layer
        self.fc = nn.Linear(hidden_size, n_output)

    def forward(self, x):

        # Preprocessing
        x = F.relu(self.bn(x))

        # Get input dimensions
        batch_size, channels, height, width = x.shape

        # Reshape for LSTM
        x = x.permute(0, 3, 2, 1)
        x = x.reshape(batch_size, width, height * channels)

        # LSTM forward pass
        lstm_out, _ = self.lstm(x)

        # Take the output from the last time step
        last_output = lstm_out[:, -1, :]

        # Pass through fully connected layer
        out = self.fc(last_output)

        return F.log_softmax(out, dim=1)

## ResNet-18
*from the article "Deep residual learning for image recognition"*


In [ ]:
class BasicResidualBlock(nn.Module):
    """
    Basic residual block for ResNet-18 with the following structure:
    - Conv 3x3 -> BN -> ReLU
    - Conv 3x3 -> BN
    - Add residual connection
    - ReLU
    """
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(
            inplanes, planes,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(
            planes, planes,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        # First conv block
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # Second conv block
        out = self.conv2(out)
        out = self.bn2(out)

        # Add residual connection
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [ ]:
class ResNet18(nn.Module):
    def __init__(
            self,
            n_output=10
        ):
        super(ResNet18, self).__init__()

        self.inplanes = 64

        # Initial convolution block
        self.conv1 = nn.Conv2d(
            in_channels = 1,
            out_channels = 64,
            kernel_size = 7,
            stride = 2,
            padding = 3,
            bias = False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers (ResNet-18 has [2, 2, 2, 2] blocks)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Calculate the feature size after all layers
        self.fc = nn.Linear(512, n_output)

        # Initialize weights
        self._initialize_weights()

    def _make_layer(self, planes, blocks, stride=1):
        """Create a residual layer with multiple BasicBlocks"""

        downsample = None

        # Constraint on when to downsamples
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

        layers = []

        # First block (may include downsampling)
        layers.append(BasicResidualBlock(self.inplanes, planes, stride, downsample))
        self.inplanes = planes

        # Remaining blocks
        for _ in range(1, blocks):
            layers.append(BasicResidualBlock(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        """Initialize weights using Kaiming initialization"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):

        # Initial conv block
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Residual layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Global pooling and classification
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

# Training and evaluation of the models

In [ ]:
# Check on the availability of GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

print("Using device:", device)

Using device: cuda


To be able to have a coherent training of the models, a class "modelTrainer" has been created and used to train all the models (also the ones in the next notebooks).

In [ ]:
class modelTrainer:
    def __init__(
            self,
            model: nn.Module,
            train_loader: DataLoader,
            val_loader: DataLoader,
            loss: nn.Module,
            optimizer: optim.Optimizer,
            num_epochs: int,
            save_dir: str = "./checkpoints",
            experiment_name: str = "experiment",
            scheduler: Optional[object] = None,
            patience: int = 20,
            l2_reg: bool = False,
            lambda_reg: float = 0.001,
            verbose: bool = True
        ):
        """Set up training process"""

        # Automatically detect device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Paramters for the training of the models
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.loss = loss
        self.optimizer = optimizer
        self.num_epochs = num_epochs
        self.scheduler = scheduler
        self.patience = patience
        self.l2_reg = l2_reg
        self.lambda_reg = lambda_reg
        self.verbose = verbose

        # Setup save directory
        self.save_dir = Path(save_dir) / experiment_name
        self.save_dir.mkdir(parents=True, exist_ok=True)

        # Training state
        self.current_epoch = 0
        self.best_val_loss = float('inf')
        self.best_val_acc = 0.0
        self.patience_counter = 0
        self.history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'learning_rates': []
        }

        if self.verbose:
            print(f"Training on device: {self.device}")
            print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")

    def compute_l2_loss(self) -> torch.Tensor:
        """Compute L2 regularization loss for convolutional layers"""
        l2_loss = torch.tensor(0., device=self.device)

        for module in self.model.modules():
            if isinstance(module, (nn.Conv1d, nn.Conv2d, nn.Conv3d)):
                # Add L2 norm of the weights
                l2_loss += torch.norm(module.weight, p=2)

        return self.lambda_reg * l2_loss

    def train_epoch(self) -> Dict[str, float]:
        """Train for one epoch using the train dataloader"""
        self.model.train()
        running_loss = 0.0
        running_classification_loss = 0.0
        running_reg_loss = 0.0
        correct = 0
        total = 0

        pbar = tqdm(self.train_loader, desc=f"Epoch {self.current_epoch+1}/{self.num_epochs} [Train]",
                    disable=not self.verbose)

        for batch_idx, (data, target) in enumerate(pbar):

            # Ensure correct input shape (batch, 1, height, width)
            if data.dim() == 3:
                data = data.unsqueeze(1)

            # Move data and target to the appropriate device
            data = data.to(self.device)
            target = target.to(self.device)

            # Forward pass
            self.optimizer.zero_grad()
            output = self.model(data)

            # Classification loss
            classification_loss = self.loss(output, target)

            # Total loss with optional L2 regularization
            if self.l2_reg:
                reg_loss = self.compute_l2_loss()
                total_loss = classification_loss + reg_loss
                running_reg_loss += reg_loss.item()
            else:
                total_loss = classification_loss

            # Backward pass
            total_loss.backward()
            self.optimizer.step()

            # Metrics
            running_loss += total_loss.item()
            running_classification_loss += classification_loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

            # Update progress bar
            if self.verbose:
                postfix = {
                    'loss': f'{running_loss/(batch_idx+1):.4f}',
                    'acc': f'{100.*correct/total:.2f}%'
                }
                if self.l2_reg:
                    postfix['reg'] = f'{running_reg_loss/(batch_idx+1):.4f}'
                pbar.set_postfix(postfix)

        epoch_loss = running_loss / len(self.train_loader)
        epoch_acc = 100. * correct / total

        metrics = {'loss': epoch_loss, 'accuracy': epoch_acc}
        if self.l2_reg:
            metrics['classification_loss'] = running_classification_loss / len(self.train_loader)
            metrics['reg_loss'] = running_reg_loss / len(self.train_loader)

        return metrics

    @torch.no_grad()
    def validate(self) -> Dict[str, float]:
        """Validate the model using the val dataloader"""
        self.model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        pbar = tqdm(self.val_loader, desc=f"Epoch {self.current_epoch+1}/{self.num_epochs} [Val]",
                    disable=not self.verbose)

        for data, target in pbar:

            if data.dim() == 3:
                data = data.unsqueeze(1)

            # Move data and target to the appropriate device
            data = data.to(self.device)
            target = target.to(self.device)

            output = self.model(data)
            val_loss += self.loss(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

            if self.verbose:
                pbar.set_postfix({
                    'loss': f'{val_loss/(pbar.n+1):.4f}',
                    'acc': f'{100.*correct/total:.2f}%'
                })

        val_loss /= len(self.val_loader)
        val_acc = 100. * correct / total

        return {'loss': val_loss, 'accuracy': val_acc}

    def save_checkpoint(self, filename: str = "checkpoint.pth", is_best: bool = False):
        """Save model checkpoint"""
        checkpoint = {
            'epoch': self.current_epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'best_val_loss': self.best_val_loss,
            'best_val_acc': self.best_val_acc,
            'history': self.history,
            'l2_reg': self.l2_reg,
            'lambda_reg': self.lambda_reg
        }

        if self.scheduler is not None:
            checkpoint['scheduler_state_dict'] = self.scheduler.state_dict()

        filepath = self.save_dir / filename
        torch.save(checkpoint, filepath)

        if is_best and self.verbose:
            print(f"Best model saved")

    def load_checkpoint(self, filename: str = "best_model.pth"):
        """Load model checkpoint"""
        filepath = self.save_dir / filename
        checkpoint = torch.load(filepath)

        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.current_epoch = checkpoint['epoch']
        self.best_val_loss = checkpoint['best_val_loss']
        self.best_val_acc = checkpoint['best_val_acc']
        self.history = checkpoint['history']

        if self.scheduler is not None and 'scheduler_state_dict' in checkpoint:
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        if self.verbose:
            print(f"Checkpoint loaded from {filepath}")

    def train(self) -> Dict[str, list]:
        """Main training loop"""
        if self.verbose:
            print(f"\n{'='*70}")
            print(f"Starting Training of the experiment: {self.save_dir.name}")
            print(f"{'='*70}\n")

        start_time = time.time()

        for epoch in range(self.num_epochs):
            self.current_epoch = epoch

            # Train
            train_metrics = self.train_epoch()

            # Validate
            val_metrics = self.validate()

            # Update history
            self.history['train_loss'].append(train_metrics['loss'])
            self.history['train_acc'].append(train_metrics['accuracy'])
            self.history['val_loss'].append(val_metrics['loss'])
            self.history['val_acc'].append(val_metrics['accuracy'])

            # Track learning rate
            current_lr = self.optimizer.param_groups[0]['lr']
            self.history['learning_rates'].append(current_lr)

            # Learning rate scheduling
            if self.scheduler is not None:
                if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                    self.scheduler.step(val_metrics['loss'])
                else:
                    self.scheduler.step()

            # Print epoch summary
            if self.verbose:
                print(f"\nEpoch {epoch+1}/{self.num_epochs} Summary:")
                print(f"Train Loss: {train_metrics['loss']:.4f} | Train Acc: {train_metrics['accuracy']:.2f}%")
                if self.l2_reg:
                    print(f"Classification: {train_metrics['classification_loss']:.4f} | Regularization: {train_metrics['reg_loss']:.4f}")
                print(f"Val Loss: {val_metrics['loss']:.4f} | Val Acc: {val_metrics['accuracy']:.2f}%")
                print(f"Learning Rate: {current_lr:.6f}")

            # Early stopping and checkpointing
            if val_metrics['loss'] < self.best_val_loss:
                self.best_val_loss = val_metrics['loss']
                self.best_val_acc = val_metrics['accuracy']
                self.patience_counter = 0
                self.save_checkpoint("best_model.pth", is_best=True)
            else:
                self.patience_counter += 1
                if self.verbose:
                    print(f"  Patience: {self.patience_counter}/{self.patience}")

                if self.patience_counter >= self.patience:
                    if self.verbose:
                        print(f"\n{'='*70}")
                        print(f"Early stopping triggered at epoch {epoch+1}")
                        print(f"{'='*70}\n")
                    break

            # Save periodic checkpoint every 10 epochs
            if (epoch + 1) % 10 == 0:
                self.save_checkpoint(f"checkpoint_epoch_{epoch+1}.pth")

        # Training complete
        elapsed_time = time.time() - start_time
        if self.verbose:
            print(f"\n{'='*70}")
            print(f"Training Completed")
            print(f"{'='*70}")
            print(f"Total time: {elapsed_time/60:.2f} minutes")
            print(f"Best validation loss: {self.best_val_loss:.4f}")
            print(f"Best validation accuracy: {self.best_val_acc:.2f}%")
            print(f"{'='*70}\n")

        # Load best model
        self.load_checkpoint("best_model.pth")

        # Save final history
        self.save_history()

        return self.history

    def save_history(self):
        """Save training history to JSON"""

        history_path = self.save_dir / "training_history.json"
        with open(history_path, 'w') as f:
            json.dump({
                'history': self.history,
                'best_val_loss': self.best_val_loss,
                'best_val_acc': self.best_val_acc,
                'total_epochs': self.current_epoch + 1,
                'l2_reg': self.l2_reg,
                'lambda_reg': self.lambda_reg
            }, f, indent=4)

        if self.verbose:
            print(f"Training history saved to {history_path}")

    @torch.no_grad()
    def evaluate(self, test_loader: DataLoader) -> Dict[str, float]:
        """Evaluate model on test set"""
        self.model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        if self.verbose:
            print("\nEvaluating on test set...")

        pbar = tqdm(test_loader, desc="Testing", disable=not self.verbose)

        for data, target in pbar:

            if data.dim() == 3:
                data = data.unsqueeze(1)

            # Move data and target to the appropriate device
            data = data.to(self.device)
            target = target.to(self.device)

            output = self.model(data)
            test_loss += self.loss(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

        test_loss /= len(test_loader)
        test_acc = 100. * correct / total

        if self.verbose:
            print(f"\nTest Results:")
            print(f"Test Loss: {test_loss:.4f}")
            print(f"Test Accuracy: {test_acc:.2f}%\n")

        return {'loss': test_loss, 'accuracy': test_acc}

If the code has been running on google collaboratory, keep the on_drive variable "True" to have the correct directory, otherwise change to "False". If run on local device the directory will be already correct.

In [ ]:
""""Directory check"""
on_drive = True

if on_drive:
    os.chdir('/content/drive/MyDrive/Thesis')
    print("Current directory:", os.getcwd())

Current directory: /content/drive/MyDrive/Thesis


## Simple CNN
from the article "Adversial Example detection by classification for deep speech recognition"

### Initialize the model

In [ ]:
# Define input shape
sample_mfcc = train_data10['MFCCs'][0]
if isinstance(sample_mfcc, np.ndarray):
    input_shape = sample_mfcc.shape
else:
    input_shape = sample_mfcc.size()
print(f"Input shape: {input_shape}")

# Define the number of labels
labels10 = list(set(train_data10["labels"]))
n_labels10 = len(labels10)
print(f"Number of outputs: {n_labels10}")

labels30 = list(set(train_data30["labels"]))
n_labels30 = len(labels30)
print(f"Number of outputs: {n_labels30}")

Input shape: torch.Size([40, 101])
Number of outputs: 10
Number of outputs: 30


In [ ]:
# Compile the model for GSCD-10
print("Compile the model for GSCD-10", "="*45)
simpleCNN_model10 = simpleCNN(input_shape = input_shape, n_output = n_labels10)
print(simpleCNN_model10)

# Compile the model for GSCD-30
print("Compile the model for GSCD-30", "="*45)
simpleCNN_model30 = simpleCNN(input_shape = input_shape, n_output = n_labels30)
print(simpleCNN_model30)

Compile the model for GSCD-10 =============================================
simpleCNN(
  (conv1): Conv2d(1, 64, kernel_size=(2, 2), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 32, kernel_size=(2, 2), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.4, inplace=False)
  (fc1): Linear(in_features=2016, out_features=128, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_featur

In [ ]:
n10_simpleCNN = count_parameters(simpleCNN_model10)
print("Number of parameters in the model for GSCD-10: %s" % n10_simpleCNN)

n30_simpleCNN = count_parameters(simpleCNN_model30)
print("Number of parameters in the model for GSCD-30: %s" % n30_simpleCNN)

Number of parameters in the model for GSCD-10: 284778
Number of parameters in the model for GSCD-30: 287358


### Training of the model

In [ ]:
"""Parameters for the model training on the clean data of GSCD-10"""
# Define the loss
simpleCNN_loss = nn.CrossEntropyLoss()

# Define the optimizer
simpleCNN_lr = 0.0001
simpleCNN_optimizer10 = optim.Adam(
    simpleCNN_model10.parameters(),
    lr = simpleCNN_lr,
    weight_decay=0.001)

# Learning rate scheduler
simpleCNN_scheduler10 = optim.lr_scheduler.ReduceLROnPlateau(
    simpleCNN_optimizer10, mode='min', factor=0.5, patience=5
)

"""Initialize training of the model"""
simpleCNN_trainer10 = modelTrainer(
    model = simpleCNN_model10,
    train_loader = train_loader10,
    val_loader = val_loader10,
    loss = simpleCNN_loss,
    optimizer = simpleCNN_optimizer10,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "simpleCNN_clean10",
    scheduler = simpleCNN_scheduler10,
    patience = 20,
    l2_reg = True,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 284,778


In [ ]:
"""Training of the model"""
simpleCNN_history10 = simpleCNN_trainer10.train()


Starting Training of the experiment: simpleCNN_clean10



Epoch 1/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 204.15it/s, loss=1.6832, acc=53.76%]



Epoch 1/100 Summary:
Train Loss: 2.0192 | Train Acc: 26.50%
Classification: 2.0068 | Regularization: 0.0124
Val Loss: 1.4526 | Val Acc: 53.76%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 204.55it/s, loss=1.2525, acc=67.10%]



Epoch 2/100 Summary:
Train Loss: 1.4612 | Train Acc: 49.39%
Classification: 1.4489 | Regularization: 0.0123
Val Loss: 1.0981 | Val Acc: 67.10%
Learning Rate: 0.000100
Best model saved


...


Epoch 98/100 Summary:
Train Loss: 0.1913 | Train Acc: 94.23%
Classification: 0.1797 | Regularization: 0.0116
Val Loss: 0.2727 | Val Acc: 91.18%
Learning Rate: 0.000003
  Patience: 9/20


Epoch 99/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 203.38it/s, loss=0.3117, acc=91.14%]



Epoch 99/100 Summary:
Train Loss: 0.1947 | Train Acc: 93.97%
Classification: 0.1831 | Regularization: 0.0116
Val Loss: 0.2733 | Val Acc: 91.14%
Learning Rate: 0.000003
  Patience: 10/20


Epoch 100/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 205.56it/s, loss=0.3123, acc=91.10%]



Epoch 100/100 Summary:
Train Loss: 0.1934 | Train Acc: 94.11%
Classification: 0.1817 | Regularization: 0.0116
Val Loss: 0.2738 | Val Acc: 91.10%
Learning Rate: 0.000003
  Patience: 11/20

Training Completed
Total time: 11.90 minutes
Best validation loss: 0.2725
Best validation accuracy: 91.05%

Checkpoint loaded from experiments_cleanData/simpleCNN_clean10/best_model.pth
Training history saved to experiments_cleanData/simpleCNN_clean10/training_history.json


In [ ]:
"""Parameters for the model training on the clean data of GSCD-30"""
# Define the loss
simpleCNN_loss = nn.CrossEntropyLoss()

# Define the optimizer
simpleCNN_lr = 0.0001
simpleCNN_optimizer30 = optim.Adam(
    simpleCNN_model30.parameters(),
    lr = simpleCNN_lr,
    weight_decay=0.001)

# Learning rate scheduler
simpleCNN_scheduler30 = optim.lr_scheduler.ReduceLROnPlateau(
    simpleCNN_optimizer30, mode='min', factor=0.5, patience=5
)

"""Initialize model trainer"""
simpleCNN_trainer30 = modelTrainer(
    model = simpleCNN_model30,
    train_loader = train_loader30,
    val_loader = val_loader30,
    loss = simpleCNN_loss,
    optimizer = simpleCNN_optimizer30,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "simpleCNN_clean30",
    scheduler = simpleCNN_scheduler30,
    patience = 20,
    l2_reg = True,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 287,358


In [ ]:
"""Training of the model"""
simpleCNN_history30 = simpleCNN_trainer30.train()


Starting Training of the experiment: simpleCNN_clean30



Epoch 1/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 203.55it/s, loss=2.0736, acc=45.98%]



Epoch 1/100 Summary:
Train Loss: 2.9494 | Train Acc: 16.55%
Classification: 2.9372 | Regularization: 0.0122
Val Loss: 2.0628 | Val Acc: 45.98%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 203.06it/s, loss=1.5746, acc=59.57%]



Epoch 2/100 Summary:
Train Loss: 2.1173 | Train Acc: 38.59%
Classification: 2.1052 | Regularization: 0.0121
Val Loss: 1.5582 | Val Acc: 59.57%
Learning Rate: 0.000100
Best model saved


...


Epoch 98/100 Summary:
Train Loss: 0.4732 | Train Acc: 85.55%
Classification: 0.4614 | Regularization: 0.0119
Val Loss: 0.4160 | Val Acc: 88.41%
Learning Rate: 0.000006
  Patience: 1/20


Epoch 99/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 200.80it/s, loss=0.4178, acc=88.76%]



Epoch 99/100 Summary:
Train Loss: 0.4686 | Train Acc: 85.70%
Classification: 0.4567 | Regularization: 0.0119
Val Loss: 0.4135 | Val Acc: 88.76%
Learning Rate: 0.000006
  Patience: 2/20


Epoch 100/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 203.42it/s, loss=0.4186, acc=88.28%]



Epoch 100/100 Summary:
Train Loss: 0.4644 | Train Acc: 85.86%
Classification: 0.4526 | Regularization: 0.0119
Val Loss: 0.4142 | Val Acc: 88.28%
Learning Rate: 0.000006
  Patience: 3/20

Training Completed
Total time: 32.93 minutes
Best validation loss: 0.4126
Best validation accuracy: 88.38%

Checkpoint loaded from experiments_cleanData/simpleCNN_clean30/best_model.pth
Training history saved to experiments_cleanData/simpleCNN_clean30/training_history.json


### Evaluation of the model

In [ ]:
# Load the pre-trained checkpoint
simpleCNN_trainer10.load_checkpoint("best_model.pth")

simpleCNN_trainer10_loss = simpleCNN_trainer10.best_val_loss
simpleCNN_trainer10_acc = simpleCNN_trainer10.best_val_acc

print(f"\nBest Validation Loss (from training): {simpleCNN_trainer10_loss:.4f}")
print(f"Best Validation Accuracy (from training): {simpleCNN_trainer10_acc:.2f}%")

"""Evaluation of the model on GSCD-10"""
simpleCNN_test_results10 = simpleCNN_trainer10.evaluate(test_loader10)

Checkpoint loaded from experiments_cleanData/simpleCNN_clean10/best_model.pth

Best Validation Loss (from training): 0.2725
Best Validation Accuracy (from training): 91.05%

Evaluating on test set...


Testing: 100%|██████████| 74/74 [00:00<00:00, 237.84it/s]


Test Results:
Test Loss: 0.3127
Test Accuracy: 90.69%



In [ ]:
# Load the pre-trained checkpoint
simpleCNN_trainer30.load_checkpoint("best_model.pth")

simpleCNN_trainer30_loss = simpleCNN_trainer30.best_val_loss
simpleCNN_trainer30_acc = simpleCNN_trainer30.best_val_acc

print(f"\nBest Validation Loss (from training): {simpleCNN_trainer30_loss:.4f}")
print(f"Best Validation Accuracy (from training): {simpleCNN_trainer30_acc:.2f}%")

"""Evaluation of the model on GSCD-30"""
simpleCNN_test_results30 = simpleCNN_trainer30.evaluate(test_loader30)

Checkpoint loaded from experiments_cleanData/simpleCNN_clean30/best_model.pth

Best Validation Loss (from training): 0.4126
Best Validation Accuracy (from training): 88.38%

Evaluating on test set...


Testing: 100%|██████████| 195/195 [00:00<00:00, 241.63it/s]


Test Results:
Test Loss: 0.4217
Test Accuracy: 87.80%



## Complex CNN
from the article "Trojaning attack on Neural Networks"

### Initialize the model

In [ ]:
# Define input shape
sample_mfcc = train_data10['MFCCs'][0]
if isinstance(sample_mfcc, np.ndarray):
    input_shape = sample_mfcc.shape
else:
    input_shape = sample_mfcc.size()
print(f"Input shape: {input_shape}")

# Define the number of labels
labels10 = list(set(train_data10["labels"]))
n_labels10 = len(labels10)
print(f"Number of outputs: {n_labels10}")

labels30 = list(set(train_data30["labels"]))
n_labels30 = len(labels30)
print(f"Number of outputs: {n_labels30}")

Input shape: torch.Size([40, 101])
Number of outputs: 10
Number of outputs: 30


In [ ]:
# Compile the model for GSCD-10
print("Compile the model for GSCD-10", "="*45)
complexCNN_model10 = complexCNN(input_shape = input_shape, n_output = n_labels10)
print(complexCNN_model10)

# Compile the model for GSCD-30
print("Compile the model for GSCD-30", "="*45)
complexCNN_model30 = complexCNN(input_shape = input_shape, n_output = n_labels30)
print(complexCNN_model30)

Compile the model for GSCD-10 =============================================
complexCNN(
  (conv1): Conv2d(1, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16640, out_features=256, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_

In [ ]:
n10_complexCNN = count_parameters(complexCNN_model10)
print("Number of parameters in the model for GSCD-10: %s" % n10_complexCNN)

n30_complexCNN = count_parameters(complexCNN_model30)
print("Number of parameters in the model for GSCD-30: %s" % n30_complexCNN)

Number of parameters in the model for GSCD-10: 7614282
Number of parameters in the model for GSCD-30: 7616862


### Training of the model

In [ ]:
"""Parameters for the model training on the clean data of GSCD-10"""
# Define the loss
complexCNN_loss = nn.CrossEntropyLoss()

# Define the optimizer
complexCNN_lr = 0.0001
complexCNN_optimizer10 = optim.Adam(
    complexCNN_model10.parameters(),
    lr = complexCNN_lr,
    weight_decay=0.001)

# Learning rate scheduler
complexCNN_scheduler10 = optim.lr_scheduler.ReduceLROnPlateau(
    complexCNN_optimizer10, mode='min', factor=0.5, patience=5
)

"""Initialize the training of the model"""
complexCNN_trainer10 = modelTrainer(
    model = complexCNN_model10,
    train_loader = train_loader10,
    val_loader = val_loader10,
    loss = complexCNN_loss,
    optimizer = complexCNN_optimizer10,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "complexCNN_clean10",
    scheduler = complexCNN_scheduler10,
    patience = 20,
    l2_reg = True,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 7,614,282


In [ ]:
"""Training of the model"""
complexCNN_history10 = complexCNN_trainer10.train()


Starting Training of the experiment: complexCNN_clean10



Epoch 1/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 87.42it/s, loss=0.4565, acc=85.29%]



Epoch 1/100 Summary:
Train Loss: 1.3082 | Train Acc: 54.99%
Classification: 1.2651 | Regularization: 0.0431
Val Loss: 0.4565 | Val Acc: 85.29%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 87.37it/s, loss=0.2873, acc=90.88%]



Epoch 2/100 Summary:
Train Loss: 0.4332 | Train Acc: 87.92%
Classification: 0.3917 | Regularization: 0.0416
Val Loss: 0.2873 | Val Acc: 90.88%
Learning Rate: 0.000100
Best model saved


...


Epoch 24/100 Summary:
Train Loss: 0.0344 | Train Acc: 99.86%
Classification: 0.0060 | Regularization: 0.0284
Val Loss: 0.2433 | Val Acc: 94.15%
Learning Rate: 0.000025
  Patience: 18/20


Epoch 25/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 86.65it/s, loss=0.2555, acc=94.54%]



Epoch 25/100 Summary:
Train Loss: 0.0321 | Train Acc: 99.90%
Classification: 0.0041 | Regularization: 0.0280
Val Loss: 0.2555 | Val Acc: 94.54%
Learning Rate: 0.000013
  Patience: 19/20


Epoch 26/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 86.79it/s, loss=0.2679, acc=94.32%]



Epoch 26/100 Summary:
Train Loss: 0.0302 | Train Acc: 99.98%
Classification: 0.0025 | Regularization: 0.0277
Val Loss: 0.2679 | Val Acc: 94.32%
Learning Rate: 0.000013
  Patience: 20/20

Early stopping triggered at epoch 26


Training Completed
Total time: 8.16 minutes
Best validation loss: 0.2192
Best validation accuracy: 93.63%

Checkpoint loaded from experiments_cleanData/complexCNN_clean10/best_model.pth
Training history saved to experiments_cleanData/complexCNN_clean10/training_history.json


In [ ]:
"""Parameters for the model training on the clean data of GSCD-30"""
# Define the loss
complexCNN_loss = nn.CrossEntropyLoss()

# Define the optimizer
complexCNN_lr = 0.0001
complexCNN_optimizer30 = optim.Adam(
    complexCNN_model30.parameters(),
    lr = complexCNN_lr,
    weight_decay=0.001)

# Learning rate scheduler
complexCNN_scheduler30 = optim.lr_scheduler.ReduceLROnPlateau(
    complexCNN_optimizer30, mode='min', factor=0.5, patience=5
)


complexCNN_trainer30 = modelTrainer(
    model = complexCNN_model30,
    train_loader = train_loader30,
    val_loader = val_loader30,
    loss = complexCNN_loss,
    optimizer = complexCNN_optimizer30,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "complexCNN_clean30",
    scheduler = complexCNN_scheduler30,
    patience = 20,
    l2_reg = True,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 7,616,862


In [ ]:
"""Training of the model"""
complexCNN_history30 = complexCNN_trainer30.train()


Starting Training of the experiment: complexCNN_clean30



Epoch 1/100 [Val]: 100%|██████████| 192/192 [00:02<00:00, 87.06it/s, loss=0.5628, acc=83.95%]



Epoch 1/100 Summary:
Train Loss: 1.6934 | Train Acc: 51.72%
Classification: 1.6533 | Regularization: 0.0402
Val Loss: 0.5628 | Val Acc: 83.95%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 192/192 [00:02<00:00, 86.84it/s, loss=0.3820, acc=89.18%]



Epoch 2/100 Summary:
Train Loss: 0.5386 | Train Acc: 85.44%
Classification: 0.4991 | Regularization: 0.0394
Val Loss: 0.3781 | Val Acc: 89.18%
Learning Rate: 0.000100
Best model saved


...


Epoch 26/100 Summary:
Train Loss: 0.0379 | Train Acc: 99.68%
Classification: 0.0123 | Regularization: 0.0256
Val Loss: 0.3080 | Val Acc: 93.11%
Learning Rate: 0.000025
  Patience: 18/20


Epoch 27/100 [Val]: 100%|██████████| 192/192 [00:02<00:00, 87.08it/s, loss=0.3347, acc=93.22%]



Epoch 27/100 Summary:
Train Loss: 0.0329 | Train Acc: 99.84%
Classification: 0.0076 | Regularization: 0.0253
Val Loss: 0.3312 | Val Acc: 93.22%
Learning Rate: 0.000013
  Patience: 19/20


Epoch 28/100 [Val]: 100%|██████████| 192/192 [00:02<00:00, 86.78it/s, loss=0.3230, acc=93.45%]



Epoch 28/100 Summary:
Train Loss: 0.0329 | Train Acc: 99.83%
Classification: 0.0078 | Regularization: 0.0250
Val Loss: 0.3197 | Val Acc: 93.45%
Learning Rate: 0.000013
  Patience: 20/20

Early stopping triggered at epoch 28


Training Completed
Total time: 24.11 minutes
Best validation loss: 0.2801
Best validation accuracy: 92.44%

Checkpoint loaded from experiments_cleanData/complexCNN_clean30/best_model.pth
Training history saved to experiments_cleanData/complexCNN_clean30/training_history.json


### Evaluation of the model

In [ ]:
# Load the pre-trained checkpoint
complexCNN_trainer10.load_checkpoint("best_model.pth")

complexCNN_trainer10_loss = complexCNN_trainer10.best_val_loss
complexCNN_trainer10_acc = complexCNN_trainer10.best_val_acc

print(f"\nBest Validation Loss (from training): {complexCNN_trainer10_loss:.4f}")
print(f"Best Validation Accuracy (from training): {complexCNN_trainer10_acc:.2f}%")

"""Evaluation of the model on GSCD-10"""
complexCNN_test_results10 = complexCNN_trainer10.evaluate(test_loader10)

Checkpoint loaded from experiments_cleanData/complexCNN_clean10/best_model.pth

Best Validation Loss (from training): 0.2192
Best Validation Accuracy (from training): 93.63%

Evaluating on test set...


Testing: 100%|██████████| 74/74 [00:00<00:00, 91.53it/s]


Test Results:
Test Loss: 0.2115
Test Accuracy: 93.40%



In [ ]:
# Load the pre-trained checkpoint
complexCNN_trainer30.load_checkpoint("best_model.pth")

complexCNN_trainer30_loss = complexCNN_trainer30.best_val_loss
complexCNN_trainer30_acc = complexCNN_trainer30.best_val_acc

print(f"\nBest Validation Loss (from training): {complexCNN_trainer30_loss:.4f}")
print(f"Best Validation Accuracy (from training): {complexCNN_trainer30_acc:.2f}%")

"""Evaluation of the model on GSCD-30"""
complexCNN_test_results30 = complexCNN_trainer30.evaluate(test_loader30)

Checkpoint loaded from experiments_cleanData/complexCNN_clean30/best_model.pth

Best Validation Loss (from training): 0.2801
Best Validation Accuracy (from training): 92.44%

Evaluating on test set...


Testing: 100%|██████████| 195/195 [00:02<00:00, 90.93it/s]


Test Results:
Test Loss: 0.2827
Test Accuracy: 92.94%



## Speech-specific LSTM
from the article "A neural attention model for speech command recognition"

### Initialize the model

In [ ]:
# Define the number of labels
labels10 = list(set(train_data10["labels"]))
n_labels10 = len(labels10)
print(f"Number of outputs: {n_labels10}")

labels30 = list(set(train_data30["labels"]))
n_labels30 = len(labels30)
print(f"Number of outputs: {n_labels30}")

Number of outputs: 10
Number of outputs: 30


In [ ]:
# Compile the model for GSCD-10
print("Compile the model for GSCD-10", "="*45)
speechSpecificLSTM_model10 = speechSpecificLSTM(n_output = n_labels10)
print(speechSpecificLSTM_model10)

# Compile the model for GSCD-30
print("Compile the model for GSCD-30", "="*45)
speechSpecificLSTM_model30 = speechSpecificLSTM(n_output = n_labels30)
print(speechSpecificLSTM_model30)

Compile the model for GSCD-10 =============================================
speechSpecificLSTM(
  (conv1): Conv2d(1, 10, kernel_size=(5, 1), stride=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(10, 1, kernel_size=(5, 1), stride=(1, 1))
  (bn2): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm1): LSTM(40, 64, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 64, batch_first=True, bidirectional=True)
  (query_proj): Linear(in_features=128, out_features=128, bias=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)
Compile the model for GSCD-30 =============================================
speechSpecificLSTM(
  (conv1): Conv2d(1, 10, kernel_size=(5, 1), stride=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e

In [ ]:
n10_speechSpecificLSTM = count_parameters(speechSpecificLSTM_model10)
print("Number of parameters in the model for GSCD-10: %s" % n10_speechSpecificLSTM)

n30_speechSpecificLSTM = count_parameters(speechSpecificLSTM_model30)
print("Number of parameters in the model for GSCD-30: %s" % n30_speechSpecificLSTM)

Number of parameters in the model for GSCD-10: 180911
Number of parameters in the model for GSCD-30: 181571


### Training of the model

In [ ]:
"""Parameters for the model training on the clean data of GSCD-10"""
# Define the loss
speechSpecificLSTM_loss = nn.CrossEntropyLoss()

# Define the optimizer
speechSpecificLSTM_lr = 0.0001
speechSpecificLSTM_optimizer10 = optim.Adam(
    speechSpecificLSTM_model10.parameters(),
    lr = speechSpecificLSTM_lr,
    weight_decay=0.001)

# Learning rate scheduler
speechSpecificLSTM_scheduler10 = optim.lr_scheduler.ReduceLROnPlateau(
    speechSpecificLSTM_optimizer10, mode='min', factor=0.5, patience=5
)

"""Initialize training of the model"""
speechSpecificLSTM_trainer10 = modelTrainer(
    model = speechSpecificLSTM_model10,
    train_loader = train_loader10,
    val_loader = val_loader10,
    loss = speechSpecificLSTM_loss,
    optimizer = speechSpecificLSTM_optimizer10,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "speechSpecificLSTM_clean10",
    scheduler = speechSpecificLSTM_scheduler10,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 180,911


In [ ]:
"""Training of the model"""
speechSpecificLSTM_history10 = speechSpecificLSTM_trainer10.train()


Starting Training of the experiment: speechSpecificLSTM_clean10



Epoch 1/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 339.05it/s, loss=2.2938, acc=24.69%]



Epoch 1/100 Summary:
Train Loss: 2.2992 | Train Acc: 10.53%
Val Loss: 2.2310 | Val Acc: 24.69%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 335.52it/s, loss=1.5522, acc=45.89%]



Epoch 2/100 Summary:
Train Loss: 1.8407 | Train Acc: 32.31%
Val Loss: 1.4884 | Val Acc: 45.89%
Learning Rate: 0.000100
Best model saved


...


Epoch 52/100 Summary:
Train Loss: 0.0887 | Train Acc: 97.39%
Val Loss: 0.3415 | Val Acc: 91.23%
Learning Rate: 0.000013
  Patience: 18/20


Epoch 53/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 332.24it/s, loss=0.3552, acc=91.01%]



Epoch 53/100 Summary:
Train Loss: 0.0879 | Train Acc: 97.41%
Val Loss: 0.3455 | Val Acc: 91.01%
Learning Rate: 0.000006
  Patience: 19/20


Epoch 54/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 327.93it/s, loss=0.3727, acc=91.31%]



Epoch 54/100 Summary:
Train Loss: 0.0868 | Train Acc: 97.46%
Val Loss: 0.3420 | Val Acc: 91.31%
Learning Rate: 0.000006
  Patience: 20/20

Early stopping triggered at epoch 54


Training Completed
Total time: 3.80 minutes
Best validation loss: 0.3079
Best validation accuracy: 90.88%

Checkpoint loaded from experiments_cleanData/speechSpecificLSTM_clean10/best_model.pth
Training history saved to experiments_cleanData/speechSpecificLSTM_clean10/training_history.json


In [ ]:
"""Parameters for the model training on the clean data of GSCD-30"""
# Define the loss
speechSpecificLSTM_loss = nn.CrossEntropyLoss()

# Define the optimizer
speechSpecificLSTM_lr = 0.0001
speechSpecificLSTM_optimizer30 = optim.Adam(
    speechSpecificLSTM_model30.parameters(),
    lr = speechSpecificLSTM_lr,
    weight_decay=0.001)

# Learning rate scheduler
speechSpecificLSTM_scheduler30 = optim.lr_scheduler.ReduceLROnPlateau(
    speechSpecificLSTM_optimizer30, mode='min', factor=0.5, patience=5
)

"""Initialize the training of the model"""
speechSpecificLSTM_trainer30 = modelTrainer(
    model = speechSpecificLSTM_model30,
    train_loader = train_loader30,
    val_loader = val_loader30,
    loss = speechSpecificLSTM_loss,
    optimizer = speechSpecificLSTM_optimizer30,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "speechSpecificLSTM_clean30",
    scheduler = speechSpecificLSTM_scheduler30,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 181,571


In [ ]:
"""Training of the model"""
speechSpecificLSTM_history30 = speechSpecificLSTM_trainer30.train()


Starting Training of the experiment: speechSpecificLSTM_clean30



Epoch 1/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 340.02it/s, loss=3.6428, acc=3.36%]



Epoch 1/100 Summary:
Train Loss: 3.3999 | Train Acc: 3.57%
Val Loss: 3.3582 | Val Acc: 3.36%
Learning Rate: 0.000100
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 342.18it/s, loss=3.2783, acc=9.39%]



Epoch 2/100 Summary:
Train Loss: 3.2019 | Train Acc: 6.74%
Val Loss: 3.0051 | Val Acc: 9.39%
Learning Rate: 0.000100
Best model saved


...


Epoch 57/100 Summary:
Train Loss: 0.1684 | Train Acc: 95.68%
Val Loss: 0.4660 | Val Acc: 90.35%
Learning Rate: 0.000025
  Patience: 18/20


Epoch 58/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 342.19it/s, loss=0.5304, acc=89.96%]



Epoch 58/100 Summary:
Train Loss: 0.1627 | Train Acc: 95.87%
Val Loss: 0.4862 | Val Acc: 89.96%
Learning Rate: 0.000013
  Patience: 19/20


Epoch 59/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 327.17it/s, loss=0.5410, acc=90.44%]



Epoch 59/100 Summary:
Train Loss: 0.1606 | Train Acc: 95.92%
Val Loss: 0.4677 | Val Acc: 90.44%
Learning Rate: 0.000013
  Patience: 20/20

Early stopping triggered at epoch 59


Training Completed
Total time: 11.37 minutes
Best validation loss: 0.4268
Best validation accuracy: 89.59%

Checkpoint loaded from experiments_cleanData/speechSpecificLSTM_clean30/best_model.pth
Training history saved to experiments_cleanData/speechSpecificLSTM_clean30/training_history.json


### Evaluation of the model

In [ ]:
# Load the pre-trained checkpoint
speechSpecificLSTM_trainer10.load_checkpoint("best_model.pth")

speechSpecificLSTM_trainer10_loss = speechSpecificLSTM_trainer10.best_val_loss
speechSpecificLSTM_trainer10_acc = speechSpecificLSTM_trainer10.best_val_acc

print(f"\nBest Validation Loss (from training): {speechSpecificLSTM_trainer10_loss:.4f}")
print(f"Best Validation Accuracy (from training): {speechSpecificLSTM_trainer10_acc:.2f}%")

"""Evaluation of the model on GSCD-10"""
speechSpecificLSTM_test_results10 = speechSpecificLSTM_trainer10.evaluate(test_loader10)

Checkpoint loaded from experiments_cleanData/speechSpecificLSTM_clean10/best_model.pth

Best Validation Loss (from training): 0.3079
Best Validation Accuracy (from training): 90.88%

Evaluating on test set...


Testing: 100%|██████████| 74/74 [00:00<00:00, 441.70it/s]


Test Results:
Test Loss: 0.3640
Test Accuracy: 90.90%



In [ ]:
# Load the pre-trained checkpoint
speechSpecificLSTM_trainer30.load_checkpoint("best_model.pth")

speechSpecificLSTM_trainer30_loss = speechSpecificLSTM_trainer30.best_val_loss
speechSpecificLSTM_trainer30_acc = speechSpecificLSTM_trainer30.best_val_acc

print(f"\nBest Validation Loss (from training): {speechSpecificLSTM_trainer30_loss:.4f}")
print(f"Best Validation Accuracy (from training): {speechSpecificLSTM_trainer30_acc:.2f}%")

"""Evaluation of the model on GSCD-30"""
speechSpecificLSTM_test_results30 = speechSpecificLSTM_trainer30.evaluate(test_loader30)

Checkpoint loaded from experiments_cleanData/speechSpecificLSTM_clean30/best_model.pth

Best Validation Loss (from training): 0.4268
Best Validation Accuracy (from training): 89.59%

Evaluating on test set...


Testing: 100%|██████████| 195/195 [00:00<00:00, 486.40it/s]


Test Results:
Test Loss: 0.4452
Test Accuracy: 89.57%



## Original LSTM
from the article "Long Short-Term memory"

### Initialize the model

In [ ]:
# Define the number of labels
labels10 = list(set(train_data10["labels"]))
n_labels10 = len(labels10)
print(f"Number of outputs: {n_labels10}")

labels30 = list(set(train_data30["labels"]))
n_labels30 = len(labels30)
print(f"Number of outputs: {n_labels30}")

Number of outputs: 10
Number of outputs: 30


In [ ]:
# Compile the model for GSCD-10
print("Compile the model for GSCD-10", "="*45)
originalLSTM_model10 = originalLSTM(n_output = n_labels10)
print(originalLSTM_model10)

# Compile the model for GSCD-30
print("Compile the model for GSCD-30", "="*45)
originalLSTM_model30 = originalLSTM(n_output = n_labels30)
print(originalLSTM_model30)

Compile the model for GSCD-10 =============================================
originalLSTM(
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(40, 64, num_layers=2, batch_first=True, dropout=0.4)
  (fc): Linear(in_features=64, out_features=10, bias=True)
)
Compile the model for GSCD-30 =============================================
originalLSTM(
  (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(40, 64, num_layers=2, batch_first=True, dropout=0.4)
  (fc): Linear(in_features=64, out_features=30, bias=True)
)


In [ ]:
n10_originalLSTM = count_parameters(originalLSTM_model10)
print("Number of parameters in the model for GSCD-10: %s" % n10_originalLSTM)

n30_originalLSTM = count_parameters(originalLSTM_model30)
print("Number of parameters in the model for GSCD-30: %s" % n30_originalLSTM)

Number of parameters in the model for GSCD-10: 61068
Number of parameters in the model for GSCD-30: 62368


### Training of the model

In [ ]:
"""Parameters for the model training on the clean data of GSCD-10"""
# Define the loss
originalLSTM_loss = nn.CrossEntropyLoss()

# Define the optimizer
originalLSTM_lr = 0.005
originalLSTM_optimizer10 = optim.SGD(
    originalLSTM_model10.parameters(),
    lr = originalLSTM_lr,
    weight_decay=0.0001,
    momentum=0.9)

# Learning rate scheduler
originalLSTM_scheduler10 = optim.lr_scheduler.ReduceLROnPlateau(
    originalLSTM_optimizer10, mode='min', factor=0.5, patience=5
)

"""Initialize training of the model"""
originalLSTM_trainer10 = modelTrainer(
    model = originalLSTM_model10,
    train_loader = train_loader10,
    val_loader = val_loader10,
    loss = originalLSTM_loss,
    optimizer = originalLSTM_optimizer10,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "originalLSTM_clean10",
    scheduler = originalLSTM_scheduler10,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 61,068


In [ ]:
"""Training of the model"""
originalLSTM_history10 = originalLSTM_trainer10.train()


Starting Training of the experiment: originalLSTM_clean10



Epoch 1/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 429.89it/s, loss=3.7338, acc=9.68%]



Epoch 1/100 Summary:
Train Loss: 2.3024 | Train Acc: 10.53%
Val Loss: 2.3017 | Val Acc: 9.68%
Learning Rate: 0.005000
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 422.29it/s, loss=3.8150, acc=12.52%]



Epoch 2/100 Summary:
Train Loss: 2.3008 | Train Acc: 10.97%
Val Loss: 2.2994 | Val Acc: 12.52%
Learning Rate: 0.005000
Best model saved


...


Epoch 82/100 Summary:
Train Loss: 0.1136 | Train Acc: 96.20%
Val Loss: 0.3088 | Val Acc: 91.31%
Learning Rate: 0.000625
  Patience: 18/20


Epoch 83/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 386.02it/s, loss=0.5658, acc=90.92%]



Epoch 83/100 Summary:
Train Loss: 0.1113 | Train Acc: 96.37%
Val Loss: 0.3178 | Val Acc: 90.92%
Learning Rate: 0.000313
  Patience: 19/20


Epoch 84/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 388.33it/s, loss=0.5522, acc=91.48%]



Epoch 84/100 Summary:
Train Loss: 0.1090 | Train Acc: 96.39%
Val Loss: 0.3101 | Val Acc: 91.48%
Learning Rate: 0.000313
  Patience: 20/20

Early stopping triggered at epoch 84


Training Completed
Total time: 3.56 minutes
Best validation loss: 0.2774
Best validation accuracy: 91.53%

Checkpoint loaded from experiments_cleanData/originalLSTM_clean10/best_model.pth
Training history saved to experiments_cleanData/originalLSTM_clean10/training_history.json


In [ ]:
"""Parameters for the model training on the clean data of GSCD-30"""
# Define the loss
originalLSTM_loss = nn.CrossEntropyLoss()

# Define the optimizer
originalLSTM_lr = 0.005
originalLSTM_optimizer30 = optim.SGD(
    originalLSTM_model30.parameters(),
    lr = originalLSTM_lr,
    weight_decay=0.0001,
    momentum=0.9)

# Learning rate scheduler
originalLSTM_scheduler30 = optim.lr_scheduler.ReduceLROnPlateau(
    originalLSTM_optimizer30, mode='min', factor=0.5, patience=5
)

"""Initiliaze training of the model"""
originalLSTM_trainer30 = modelTrainer(
    model = originalLSTM_model30,
    train_loader = train_loader30,
    val_loader = val_loader30,
    loss = originalLSTM_loss,
    optimizer = originalLSTM_optimizer30,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "originalLSTM_clean30",
    scheduler = originalLSTM_scheduler30,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 62,368


In [ ]:
"""Training of the model"""
originalLSTM_history30 = originalLSTM_trainer30.train()


Starting Training of the experiment: originalLSTM_clean30



Epoch 1/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 403.00it/s, loss=3.8911, acc=4.95%]



Epoch 1/100 Summary:
Train Loss: 3.3931 | Train Acc: 3.62%
Val Loss: 3.3844 | Val Acc: 4.95%
Learning Rate: 0.005000
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 398.19it/s, loss=3.8869, acc=5.08%]



Epoch 2/100 Summary:
Train Loss: 3.3900 | Train Acc: 4.17%
Val Loss: 3.3808 | Val Acc: 5.08%
Learning Rate: 0.005000
Best model saved


...


Epoch 98/100 Summary:
Train Loss: 0.1489 | Train Acc: 95.74%
Val Loss: 0.3246 | Val Acc: 91.05%
Learning Rate: 0.000156
  Patience: 7/20


Epoch 99/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 395.77it/s, loss=0.3828, acc=91.12%]



Epoch 99/100 Summary:
Train Loss: 0.1473 | Train Acc: 95.79%
Val Loss: 0.3230 | Val Acc: 91.12%
Learning Rate: 0.000156
  Patience: 8/20


Epoch 100/100 [Val]: 100%|██████████| 192/192 [00:00<00:00, 395.85it/s, loss=0.3800, acc=91.07%]



Epoch 100/100 Summary:
Train Loss: 0.1480 | Train Acc: 95.79%
Val Loss: 0.3187 | Val Acc: 91.07%
Learning Rate: 0.000156
Best model saved

Training Completed
Total time: 11.70 minutes
Best validation loss: 0.3187
Best validation accuracy: 91.07%

Checkpoint loaded from experiments_cleanData/originalLSTM_clean30/best_model.pth
Training history saved to experiments_cleanData/originalLSTM_clean30/training_history.json


### Evaluation of the model

In [ ]:
# Load the pre-trained checkpoint
originalLSTM_trainer10.load_checkpoint("best_model.pth")

originalLSTM_trainer10_loss = originalLSTM_trainer10.best_val_loss
originalLSTM_trainer10_acc = originalLSTM_trainer10.best_val_acc

print(f"\nBest Validation Loss (from training): {originalLSTM_trainer10_loss:.4f}")
print(f"Best Validation Accuracy (from training): {originalLSTM_trainer10_acc:.2f}%")

"""Evaluation of the model on GSCD-10"""
originalLSTM_test_results10 = originalLSTM_trainer10.evaluate(test_loader10)

Checkpoint loaded from experiments_cleanData/originalLSTM_clean10/best_model.pth

Best Validation Loss (from training): 0.2774
Best Validation Accuracy (from training): 91.53%

Evaluating on test set...


Testing: 100%|██████████| 74/74 [00:00<00:00, 559.75it/s]


Test Results:
Test Loss: 0.2986
Test Accuracy: 90.65%



In [ ]:
# Load the pre-trained checkpoint
originalLSTM_trainer30.load_checkpoint("best_model.pth")

originalLSTM_trainer30_loss = originalLSTM_trainer30.best_val_loss
originalLSTM_trainer30_acc = originalLSTM_trainer30.best_val_acc

print(f"\nBest Validation Loss (from training): {originalLSTM_trainer30_loss:.4f}")
print(f"Best Validation Accuracy (from training): {originalLSTM_trainer30_acc:.2f}%")

"""Evaluation of the model on GSCD-30"""
originalLSTM_test_results30 = originalLSTM_trainer30.evaluate(test_loader30)

Checkpoint loaded from experiments_cleanData/originalLSTM_clean30/best_model.pth

Best Validation Loss (from training): 0.3187
Best Validation Accuracy (from training): 91.07%

Evaluating on test set...


Testing: 100%|██████████| 195/195 [00:00<00:00, 588.31it/s]


Test Results:
Test Loss: 0.3351
Test Accuracy: 90.71%



## ResNet-18
from the article "Deep residual learning for image recognition"


### Initialize the model

In [ ]:
# Define the number of labels
labels10 = list(set(train_data10["labels"]))
n_labels10 = len(labels10)
print(f"Number of outputs: {n_labels10}")

labels30 = list(set(train_data30["labels"]))
n_labels30 = len(labels30)
print(f"Number of outputs: {n_labels30}")

Number of outputs: 10
Number of outputs: 30


In [ ]:
# Compile the model for GSCD-10
print("Compile the model for GSCD-10", "="*45)
ResNet18_model10 = ResNet18(n_output = n_labels10)
print(ResNet18_model10)

# Compile the model for GSCD-30
print("Compile the model for GSCD-30", "="*45)
ResNet18_model30 = ResNet18(n_output = n_labels30)
print(ResNet18_model30)

Compile the model for GSCD-10 =============================================
ResNet18(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1

In [ ]:
n10_ResNet18 = count_parameters(ResNet18_model10)
print("Number of parameters in the model for GSCD-10: %s" % n10_ResNet18)

n30_ResNet18 = count_parameters(ResNet18_model30)
print("Number of parameters in the model for GSCD-30: %s" % n30_ResNet18)

Number of parameters in the model for GSCD-10: 11175370
Number of parameters in the model for GSCD-30: 11185630


### Training of the model

In [ ]:
"""Parameters for the model training on the clean data of GSCD-10"""
# Define the loss
ResNet18_loss = nn.CrossEntropyLoss()

# Define the optimizer
ResNet18_lr = 0.01
ResNet18_optimizer10 = optim.SGD(
    ResNet18_model10.parameters(),
    lr = ResNet18_lr,
    weight_decay=0.0001,
    momentum=0.9)

# Learning rate scheduler
ResNet18_scheduler10 = optim.lr_scheduler.ReduceLROnPlateau(
    ResNet18_optimizer10, mode='min', factor=0.5, patience=5
)

"""Initialize training of the model"""
ResNet18_trainer10 = modelTrainer(
    model = ResNet18_model10,
    train_loader = train_loader10,
    val_loader = val_loader10,
    loss = ResNet18_loss,
    optimizer = ResNet18_optimizer10,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "ResNet18_clean10",
    scheduler = ResNet18_scheduler10,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 11,175,370


In [ ]:
"""Training of the model"""
ResNet18_history10 = ResNet18_trainer10.train()


Starting Training of the experiment: ResNet18_clean10



Epoch 1/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 146.35it/s, loss=1.0509, acc=73.03%]



Epoch 1/100 Summary:
Train Loss: 0.8917 | Train Acc: 68.83%
Val Loss: 0.9214 | Val Acc: 73.03%
Learning Rate: 0.010000
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 146.86it/s, loss=0.3593, acc=89.89%]



Epoch 2/100 Summary:
Train Loss: 0.3438 | Train Acc: 88.50%
Val Loss: 0.3052 | Val Acc: 89.89%
Learning Rate: 0.010000
Best model saved


...


Epoch 25/100 Summary:
Train Loss: 0.0005 | Train Acc: 99.99%
Val Loss: 0.2623 | Val Acc: 94.67%
Learning Rate: 0.002500
  Patience: 18/20


Epoch 26/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 148.38it/s, loss=0.3011, acc=94.67%]



Epoch 26/100 Summary:
Train Loss: 0.0004 | Train Acc: 100.00%
Val Loss: 0.2557 | Val Acc: 94.67%
Learning Rate: 0.001250
  Patience: 19/20


Epoch 27/100 [Val]: 100%|██████████| 73/73 [00:00<00:00, 149.07it/s, loss=0.3039, acc=94.71%]



Epoch 27/100 Summary:
Train Loss: 0.0004 | Train Acc: 100.00%
Val Loss: 0.2581 | Val Acc: 94.71%
Learning Rate: 0.001250
  Patience: 20/20

Early stopping triggered at epoch 27


Training Completed
Total time: 4.89 minutes
Best validation loss: 0.2218
Best validation accuracy: 92.52%

Checkpoint loaded from experiments_cleanData/ResNet18_clean10/best_model.pth
Training history saved to experiments_cleanData/ResNet18_clean10/training_history.json


In [ ]:
"""Parameters for the model training on the clean data of GSCD-30"""
# Define the loss
ResNet18_loss = nn.CrossEntropyLoss()

# Define the optimizer
ResNet18_lr = 0.01
ResNet18_optimizer30 = optim.SGD(
    ResNet18_model30.parameters(),
    lr = ResNet18_lr,
    weight_decay=0.0001,
    momentum=0.9)

# Learning rate scheduler
ResNet18_scheduler30 = optim.lr_scheduler.ReduceLROnPlateau(
    ResNet18_optimizer30, mode='min', factor=0.5, patience=5
)

"""Initialize training of the model"""
ResNet18_trainer30 = modelTrainer(
    model = ResNet18_model30,
    train_loader = train_loader30,
    val_loader = val_loader30,
    loss = ResNet18_loss,
    optimizer = ResNet18_optimizer30,
    num_epochs = 100,
    save_dir = "./experiments_cleanData",
    experiment_name = "ResNet18_clean30",
    scheduler = ResNet18_scheduler30,
    patience = 20,
    l2_reg = False,
    lambda_reg = 0.001,
    verbose = True
)

Training on device: cuda
Model parameters: 11,185,630


In [ ]:
"""Training of the model"""
ResNet18_history30 = ResNet18_trainer30.train()


Starting Training of the experiment: ResNet18_clean30



Epoch 1/100 [Val]: 100%|██████████| 192/192 [00:01<00:00, 146.77it/s, loss=0.6058, acc=83.15%]



Epoch 1/100 Summary:
Train Loss: 0.9969 | Train Acc: 69.65%
Val Loss: 0.5774 | Val Acc: 83.15%
Learning Rate: 0.010000
Best model saved


Epoch 2/100 [Val]: 100%|██████████| 192/192 [00:01<00:00, 149.35it/s, loss=0.3787, acc=88.45%]



Epoch 2/100 Summary:
Train Loss: 0.3396 | Train Acc: 89.61%
Val Loss: 0.3689 | Val Acc: 88.45%
Learning Rate: 0.010000
Best model saved


...


Epoch 24/100 Summary:
Train Loss: 0.0005 | Train Acc: 100.00%
Val Loss: 0.2556 | Val Acc: 94.62%
Learning Rate: 0.002500
  Patience: 18/20


Epoch 25/100 [Val]: 100%|██████████| 192/192 [00:01<00:00, 148.57it/s, loss=0.2665, acc=94.72%]



Epoch 25/100 Summary:
Train Loss: 0.0004 | Train Acc: 100.00%
Val Loss: 0.2526 | Val Acc: 94.72%
Learning Rate: 0.001250
  Patience: 19/20


Epoch 26/100 [Val]: 100%|██████████| 192/192 [00:01<00:00, 148.40it/s, loss=0.2688, acc=94.64%]



Epoch 26/100 Summary:
Train Loss: 0.0004 | Train Acc: 100.00%
Val Loss: 0.2576 | Val Acc: 94.64%
Learning Rate: 0.001250
  Patience: 20/20

Early stopping triggered at epoch 26


Training Completed
Total time: 12.94 minutes
Best validation loss: 0.2405
Best validation accuracy: 93.40%

Checkpoint loaded from experiments_cleanData/ResNet18_clean30/best_model.pth
Training history saved to experiments_cleanData/ResNet18_clean30/training_history.json


### Evaluation of the model

In [ ]:
# Load the pre-trained checkpoint
ResNet18_trainer10.load_checkpoint("best_model.pth")

ResNet18_trainer10_loss = ResNet18_trainer10.best_val_loss
ResNet18_trainer10_acc = ResNet18_trainer10.best_val_acc

print(f"\nBest Validation Loss (from training): {ResNet18_trainer10_loss:.4f}")
print(f"Best Validation Accuracy (from training): {ResNet18_trainer10_acc:.2f}%")

"""Evaluation of the model on GSCD-10"""
ResNet18_test_results10 = ResNet18_trainer10.evaluate(test_loader10)

Checkpoint loaded from experiments_cleanData/ResNet18_clean10/best_model.pth

Best Validation Loss (from training): 0.2218
Best Validation Accuracy (from training): 92.52%

Evaluating on test set...


Testing: 100%|██████████| 74/74 [00:00<00:00, 162.17it/s]


Test Results:
Test Loss: 0.2395
Test Accuracy: 92.17%



In [ ]:
# Load the pre-trained checkpoint
ResNet18_trainer30.load_checkpoint("best_model.pth")

ResNet18_trainer30_loss = ResNet18_trainer30.best_val_loss
ResNet18_trainer30_acc = ResNet18_trainer30.best_val_acc

print(f"\nBest Validation Loss (from training): {ResNet18_trainer30_loss:.4f}")
print(f"Best Validation Accuracy (from training): {ResNet18_trainer30_acc:.2f}%")

"""Evaluation of the model on GSCD-30"""
ResNet18_test_results30 = ResNet18_trainer30.evaluate(test_loader30)

Checkpoint loaded from experiments_cleanData/ResNet18_clean30/best_model.pth

Best Validation Loss (from training): 0.2405
Best Validation Accuracy (from training): 93.40%

Evaluating on test set...


Testing: 100%|██████████| 195/195 [00:01<00:00, 163.12it/s]


Test Results:
Test Loss: 0.2445
Test Accuracy: 92.86%



# Choosing the best models

In [ ]:
# Best 2 models for the dataset GSCD-10
models_data10 = {
    'SimpleCNN_10': (simpleCNN_test_results10, simpleCNN_model10),
    'ComplexCNN_10': (complexCNN_test_results10, complexCNN_model10),
    'SpeechSpecificLSTM_10': (speechSpecificLSTM_test_results10, speechSpecificLSTM_model10),
    'OriginalLSTM_10': (originalLSTM_test_results10, originalLSTM_model10),
    'ResNet18_10': (ResNet18_test_results10, ResNet18_model10)
}

comparison10 = pd.DataFrame({
    name: {
        'Accuracy (%)': results['accuracy'],
        'Loss': results['loss'],
        'Parameters': count_parameters(model)
    }
    for name, (results, model) in models_data10.items()
}).T

print(comparison10.sort_values(['Accuracy (%)', 'Parameters'], ascending = [False, True]))

                       Accuracy (%)      Loss  Parameters
ComplexCNN_10             93.398223  0.211524   7614282.0
ResNet18_10               92.170969  0.239470  11175370.0
SpeechSpecificLSTM_10     90.901397  0.364048    180911.0
SimpleCNN_10              90.689801  0.312709    284778.0
OriginalLSTM_10           90.647482  0.298647     61068.0


In [ ]:
# Best 2 models for the dataset GSCD-30
models_data30 = {
    'SimpleCNN_30': (simpleCNN_test_results30, simpleCNN_model30),
    'ComplexCNN_30': (complexCNN_test_results10, complexCNN_model10),
    'SpeechSpecificLSTM_30': (speechSpecificLSTM_test_results30, speechSpecificLSTM_model30),
    'OriginalLSTM_30': (originalLSTM_test_results30, originalLSTM_model30),
    'ResNet18_30': (ResNet18_test_results30, ResNet18_model30)
}

comparison30 = pd.DataFrame({
    name: {
        'Accuracy (%)': results['accuracy'],
        'Loss': results['loss'],
        'Parameters': count_parameters(model)
    }
    for name, (results, model) in models_data30.items()
}).T

print(comparison30.sort_values(['Accuracy (%)', 'Parameters'], ascending = [False, True]))

                       Accuracy (%)      Loss  Parameters
ComplexCNN_30             93.398223  0.211524   7614282.0
ResNet18_30               92.858289  0.244473  11185630.0
OriginalLSTM_30           90.707752  0.335118     62368.0
SpeechSpecificLSTM_30     89.568288  0.445237    181571.0
SimpleCNN_30              87.802921  0.421688    287358.0


As we can see, the best two models among the ones tested are complexCNN and ResNet-18, probably because they are deeper with respect to the other models, so they can capture more features.